# <span style="color:Purple">**Bicycle Rental Management System**</span>

<span style="float: right; opacity: 0.6; color:orange">_24COP501 Programming for Specialist Applications_</span>  
<span style="float: right; opacity: 0.6; color:Orange">_By Afroz Samee, Student ID: F418164_</span>


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, HTML
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as fm
import matplotlib.animation as animation
from matplotlib.collections import PathCollection

# Import program classes
from bikeSearch import BikeSearch
from bikeRent import BicycleRentalSystem
from bikeReturn import BicycleReturnSystem
from bikeSelect import BicycleSelectionSystem,BikeRecommendationSystem,display_graphs

#### *Styling*

In [3]:
# Define a custom CSS style for your widgets
custom_style = """
<style>
    .widget-dropdown {
        color: #ffffff;  /* Text color */
        background-color: #007acc; /* Background color */
        border-radius: 5px; /* Rounded corners */
    }
    .widget-combobox {
        
        background-color: #007acc; /* Background color */
        border-radius: 5px; /* Rounded corners */
    }
    .widget-button {
        color: #ffffff;  /* Text color */
        background-color: #ff5722; /* Button background color */
        border-radius: 5px; /* Rounded corners */
    }
    .widget-text, .widget-int-text {
        background-color: #e7e7e7; /* Input background color */
        border: 1px solid #007acc; /* Border color */
    }
    .widget-output {
        border: 1px solid #007acc; /* Output border color */
        padding: 10px;
        border-radius: 5px;
        background-color: #f9f9f9; /* Output background color */
    }
    .header {
        font-size: 24px;
        color: #007acc;  /* Header color */
        font-weight: bold;
        text-align: center;
    }
    /* Information Text Styling */
    .info-text {
        font-size: 14px;
        color: #555555;  /* Text color */
        margin-top: 5px;
        margin-bottom: 5px;
        line-height: 1.6;
    }
</style>
"""

# Display the custom styles
display(HTML(custom_style))

#### *Searching Widget*

In [4]:
# Instantiate the BikeSearch class to access unique values
bike_search = BikeSearch()

class AppDisplaySearchingBikes:
    def __init__(self):
        # Text and dropdown elements
        self.dropdown_search_type = widgets.Dropdown(
            options=["Brand", "Type", "Frame_Size"],  # Options match database columns
            description="Search by:"
        )
        self.dropdown_search_term = widgets.Combobox(placeholder = 'Choose',
                                                     description="Select Value:",
                                                     ensure_option = True,
                                                     disabled = False)  # Dynamically updated dropdown
        self.btn_search_value = widgets.Button(description="Search")
        self.output = widgets.Output()

        # Set up event handlers
        self.dropdown_search_type.observe(self.on_search_type_change, names='value')
        self.btn_search_value.on_click(self.on_search_button_clicked)

        # Layout
        self.layout = widgets.VBox([
            widgets.HTML('<div class="header">Search Bicycles</div>'),
            widgets.HBox([self.dropdown_search_type, self.dropdown_search_term, self.btn_search_value]),
            self.output
        ])

        display(self.layout)

    def on_search_type_change(self, change):
        """Update the dropdown_search_term options based on selected search_type."""
        search_type = self.dropdown_search_type.value.lower()  # Get selected type

        # Set dropdown options based on search_type selection
        if search_type == "brand":
           self.dropdown_search_term.options = bike_search.brands
        elif search_type == "type":
            self.dropdown_search_term.options = bike_search.types
        elif search_type == "frame_size":
            self.dropdown_search_term.options = bike_search.frame_sizes
            

    def on_search_button_clicked(self, btn):
        """Search for bicycles and display the results."""
        with self.output:
            clear_output()  # Clear previous output
            
            search_term = self.dropdown_search_term.value
            search_type = self.dropdown_search_type.value.lower()  # Convert to lowercase to match database column
            
            # Use the bike_search instance to call the method
            df = bike_search.search_bicycles(search_term, search_type)
            
            # Display results
            if df.empty:
                display(HTML("""
                    <div class='info-text' style='font-size: 16px; padding: 10px; background-color: #f4f4f9; border-left: 4px solid #007acc; margin: 10px 0; border-radius: 5px;'>
                        <p>No bicycles found matching the search you have entered</p>
                    </div>
                    """))
            else:
                if not df.index.empty:
                    df.index = df.index.to_series().apply(
                        lambda url: f'<a href="{url}" target="_blank"><img src="{url}" style="width:50px;height:50px;"></a>'
                    )
                print("Bicycles found:")
                display(HTML(df.to_html(escape=False, index=True)))

#### *Renting Widget*

In [5]:
bike_rental_system =  BicycleRentalSystem()
class AppDisplayRentingBikes:
    def __init__(self):
        self.text_member_id = widgets.Text(description="Member ID:",
                                                            style={'description_width': 'initial'},
                                                            placeholder='Enter the Member ID here...')
        self.text_bicycle_id = widgets.Text(description="Bicycle ID:",
                                                            style={'description_width': 'initial'},
                                                            placeholder='Enter the Bicycle ID here...')
        self.inttext_rental_days = widgets.IntText(description="Rental Days:",
                                                            value=1,
                                                            style={'description_width': 'initial'},
                                                            placeholder='Enter Rental days if more than 1 day...')
        self.btn_rent_value = widgets.Button(description="Rent")
        self.output = widgets.Output()

        self.btn_rent_value.on_click(self.on_rent_button_clicked)

        # Layout
        self.layout = widgets.VBox([
            widgets.HTML('<div class="header">Rent a Bicycle</div>'),
            widgets.HBox([self.text_member_id, self.text_bicycle_id, self.inttext_rental_days, self.btn_rent_value]),
            self.output
        ])
        display(self.layout)
        

    # Event handler for rent button
    def on_rent_button_clicked(self,btn):
        with self.output:
            clear_output()
            result = bike_rental_system.rent_bicycle(self.text_member_id.value, self.text_bicycle_id.value, self.inttext_rental_days.value)
            display(Markdown(result))
            

#### *Returning Widget*

In [6]:
bike_return_system =  BicycleReturnSystem()
class AppDisplayReturningBikes:
    def __init__(self):
        self.text_bicycle_id = widgets.Text(description="Bicycle ID:",
                                                            style={'description_width': 'initial'},
                                                            placeholder='Enter the Bicycle ID here...')
        self.floattext_damage_charge = widgets.FloatText(description="Damage Charge:",
                                                            value=0.0,
                                                            style={'description_width': 'initial'},
                                                            placeholder='Enter damage charge if any...')
        self.text_damage_note = widgets.Text(description="Damage Note:",
                                                            style={'description_width': 'initial'},
                                                            placeholder='Enter any damage notes here...')
        self.btn_return_value = widgets.Button(description="Return")
        self.output = widgets.Output()

        self.btn_return_value.on_click(self.on_return_button_clicked)
        # Layout
        self.layout = widgets.VBox([
            widgets.HTML('<div class="header">Return a Bicycle</div>'),
            widgets.VBox([self.text_bicycle_id, self.floattext_damage_charge, self.text_damage_note, self.btn_return_value]),
            self.output
        ])
        display(self.layout)

    # Event handler for return button
    def on_return_button_clicked(self, btn):
        with self.output:
            clear_output()
            
            verified = bike_return_system.id_verification(self.text_bicycle_id.value)
            if verified:
                result = bike_return_system.process_return(self.text_bicycle_id.value, self.floattext_damage_charge.value, self.text_damage_note.value)
                display(Markdown(result)) 
            else:
                display(HTML("""
                <div class='info-text' style='font-size: 16px; padding: 10px; background-color: #f4f4f9; border-left: 4px solid #007acc; margin: 10px 0; border-radius: 5px;'>
                    <p>Return failed: Bicycle not verified or not rented.</p>
                </div>
                """))

#### *Selecting Widget*

In [7]:
bike_select_run = BicycleSelectionSystem()
bike_select_system =  BikeRecommendationSystem()
bike_display_graphs = display_graphs()
class AppDisplayRecommendingBikes:
    def __init__(self):
        self.inttext_budget = widgets.IntText(description="Budget:",
                                                            style={'description_width': 'initial'},
                                                            placeholder='Please enter your budget here...')
        self.inttext_replace = widgets.IntText(description="Replace Units:",
                                                            style={'description_width': 'initial'},
                                                            placeholder='Please enter number of bikes to replace...',
                                                            value = 5)
        self.inttext_retrive = widgets.IntText(description="Retrival Units:",
                                                            style={'description_width': 'initial'},
                                                            placeholder='Please enter number of top performing bikes...',
                                                            value = 5)
        self.btn_Recommend_value = widgets.Button(description="Recommend")
        self.btn_BikeReplacement_value = widgets.Button(description="Recommend")
        self.btn_BestBikes_value = widgets.Button(description="Recommend")
        self.output = widgets.Output()

        self.btn_Recommend_value.on_click(self.on_Recommend_button_clicked)
        self.btn_BikeReplacement_value.on_click(self.on_BikeReplacement_button_clicked)
        self.btn_BestBikes_value.on_click(self.on_BestBikes_button_clicked)

        # Tab layout
        self.recommend_tab_content = widgets.VBox([self.inttext_budget, self.btn_Recommend_value])
        self.replace_tab_content = widgets.VBox([self.inttext_replace, self.btn_BikeReplacement_value])
        self.best_bikes_tab_content = widgets.VBox([self.inttext_retrive, self.btn_BestBikes_value])
        
        # Create tabs
        tab_contents = [self.recommend_tab_content, self.replace_tab_content, self.best_bikes_tab_content]
        tab_labels = ['New Purchase', 'Replace Old Bike', 'Top Rented Bikes']
        
        self.tabs = widgets.Tab(children=tab_contents)
        
        # Set tab labels
        for i in range(len(tab_labels)):
            self.tabs.set_title(i, tab_labels[i])

        # Final layout including tabs and output area
        self.layout = widgets.VBox([widgets.HTML('<div class="header">Bicycle Recommendation</div>'),
                                    self.tabs,
                                    self.output])
        
        display(self.layout)

    # Event handler for return button
    def on_Recommend_button_clicked(self, btn):
        with self.output:
            clear_output()

            best_past_rentouts, best_message = bike_select_system.generate_goodrecommendations()
            
            recommend_dataframe, amount_spent, recommend_message = bike_select_system.filter_future_recommendations(best_past_rentouts,self.inttext_budget.value)
            if not recommend_dataframe.empty:
                display(Markdown(recommend_message))
                # Apply transformation to the index (ImageURL)
                if not recommend_dataframe.index.empty:
                    recommend_dataframe.index = recommend_dataframe.index.to_series().apply(
                        lambda url: f'<a href="{url}" target="_blank"><img src="{url}" style="width:50px;height:50px;"></a>'
                    )

                # Display DataFrame with clickable images in index
                display(HTML(recommend_dataframe.to_html(escape=False, index=True)))  # Use index=True to include the index in the output

                amount_spent = f"Estimated total amount for recommendated bikes: £{amount_spent}"
                display(Markdown(amount_spent))
                animation = bike_display_graphs.plot_animated_future_recommendations(self.inttext_budget.value)
                if animation: 
                    display(HTML(f"""
                                <div class='info-text' style='font-size: 16px; padding: 10px; background-color: #f4f4f9; border-left: 4px solid #007acc; margin: 10px 0; border-radius: 5px;'>
                                    <p><strong>Projected Bicycle Rental Revenue Analysis</strong></p>
                                    <p>The <strong>Projected Bicycle Rental Revenue Analysis</strong> chart provides an animated forecast of rental revenue based on the input budget of £{self.inttext_budget.value}. 
                                    This plot visualizes potential future rentals, showing how different bicycle options might contribute to revenue generation over time.</p>
                                </div>
                                """))
                    display(animation)
                else:
                    print("Error in displaying graph")
            else:
                print("Recommend failed:Error in the code or no recommendations found.")

    def on_BikeReplacement_button_clicked(self, btn):
        with self.output:
            clear_output()
            
            worst_past_rentouts, worst_message = bike_select_system.generate_badrecommendations(self.inttext_replace.value)
            
            if not worst_past_rentouts.empty:
                display(Markdown(worst_message))
                # Apply transformation to the index (ImageURL) if it's set as index
                if not worst_past_rentouts.index.empty:
                    worst_past_rentouts.index = worst_past_rentouts.index.to_series().apply(
                        lambda url: f'<a href="{url}" target="_blank"><img src="{url}" style="width:50px;height:50px;"></a>'
                    )

                # Display DataFrame with clickable images in index
                display(HTML(worst_past_rentouts.to_html(escape=False, index=True)))  # Use index=True to include the index in the output
                
                
                display(HTML("""
                <div class='info-text' style='font-size: 16px; padding: 10px; background-color: #f4f4f9; border-left: 4px solid #007acc; margin: 10px 0; border-radius: 5px;'>
                    <p><strong>Understanding the Durability vs. Rental Frequency Plot</strong></p>
                    <p>This chart illustrates bicycle recommendations by plotting <strong>Rental Frequency</strong> against the <strong>Durability Score</strong> for each model. 
                    It categorizes bicycles into two groups:</p>
                    <ul>
                        <li><strong>Good Recommendations</strong>: Bikes with high durability and rental frequency, suggesting their suitability for continued rentals.</li>
                        <li><strong>Replacement Recommendations</strong>: Bikes with lower durability or rental frequency, indicating potential for replacement or upgrade.</li>
                    </ul>
                    <p>By examining this chart, you can quickly assess which bikes are likely to offer long-term value and which may benefit from replacement to optimize rental performance.</p>
                </div>
                """))


                graph = bike_display_graphs.plot_score_durability_vs_rental_frequency()   
                     
            else:
                print("Replacement Recommend failed:Error in the code or no recommendations found.")

    def on_BestBikes_button_clicked(self, btn):
        with self.output:
            clear_output()
            
            best_past_rentouts, best_message = bike_select_system.generate_goodrecommendations(self.inttext_retrive.value)
            
            if not best_past_rentouts.empty:
                display(Markdown(best_message))
                # Apply transformation to the index (ImageURL) if it's set as index
                if not best_past_rentouts.index.empty:
                    best_past_rentouts.index = best_past_rentouts.index.to_series().apply(
                        lambda url: f'<a href="{url}" target="_blank"><img src="{url}" style="width:50px;height:50px;"></a>'
                    )
                display(HTML(best_past_rentouts.to_html(escape=False, index=True)))  # Use index=True to include the index in the output
                # Display DataFrame with clickable images in index
                
                display(HTML("""
                                <div class='info-text' style='font-size: 16px; padding: 10px; background-color: #f4f4f9; border-left: 4px solid #ff5722; margin: 10px 0; border-radius: 5px;'>
                                    <p><strong>Understanding the Brand Popularity Graph</strong></p>
                                    <p>This graph highlights the <strong>popularity of various bicycle brands</strong> based on their rental performance. By visualizing rental data across brands, 
                                    this chart reveals:</p>
                                    <ul>
                                        <li><strong>Top-performing brands</strong> that are highly preferred by users and have higher rental counts.</li>
                                        <li><strong>Less popular brands</strong> that may see lower demand, which can inform decisions on inventory adjustments.</li>
                                    </ul>
                                    <p>Use this graph to better understand brand trends and preferences, helping to align your bicycle inventory with user demand.</p>
                                </div>
                                """))

                graph = bike_display_graphs.plot_popularity_by_brand()
                
            else:
                print("Best performing Bikes Recommend failed:Error in the code or no recommendations found.")
    

#### *Bike Management System - Graphical User Interface*

In [8]:
def on_menu_change(change):
    with output:
        clear_output()
        if change['new'] == "Search Bicycles":
            app = AppDisplaySearchingBikes()
        if change['new'] == "Rent a Bicycle":
            app = AppDisplayRentingBikes()
        if change['new'] == "Return a Bicycle":
            app = AppDisplayReturningBikes()
        if change['new'] == "Bicycle Recommendation":
            app = AppDisplayRecommendingBikes()

menu_dropdown = widgets.Dropdown(options=["Select Option", "Search Bicycles", "Rent a Bicycle", "Return a Bicycle", "Bicycle Recommendation"],
                                    description="Menu:",
                                    style={'description_width': 'initial'},
                                    layout={'width': '50%'}
                                )
menu_dropdown.observe(on_menu_change, names='value')

output = widgets.Output()
display(menu_dropdown, output)

Dropdown(description='Menu:', layout=Layout(width='50%'), options=('Select Option', 'Search Bicycles', 'Rent a…

Output()